## Random Parisian Rides | Real Time Data on Vélib Stations
***

### 1 - Import libraries

In [1]:
import json
import requests
import webbrowser
import pandas as pd

import folium
from folium import IFrame
from folium.plugins import TimestampedGeoJson, MarkerCluster, LocateControl

### 2 - Data

In [23]:
information_url = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json"
status_url = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"

# Request static station data: caractéristiques et localisation des stations Vélib

r_1 = requests.get(information_url)
data_static = r_1.json()

# Request dynamic station data: le nombre de vélos et de bornettes disponibles par station

r_2 = requests.get(status_url)
data_dynamic = r_2.json()

#### Extract nested values from a JSON tree function

In [6]:
def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

#### Extract data from Vélib API

In [24]:
# Extract static Vélib data

static_datapoints = ['station_id', 'name', 'lat', 'lon', 'capacity', 'stationCode']
velib_df = pd.DataFrame()

for value in static_datapoints:
    velib_df[value] = json_extract(data_static, value)
    
velib_df.head()

,station_id,name,lat,lon,capacity,stationCode
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107
1,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104
2,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020
3,37815204,Mairie du 12ème,48.840855,2.387555,30,12109
4,100769544,Harpe - Saint-Germain,48.851519,2.343670,46,5001


In [25]:
# Extract dynamic Vélib data

dynamic_datapoints = ['station_id', 'num_bikes_available', 'numBikesAvailable',
                        'mechanical', 'ebike', 'num_docks_available', 'numDocksAvailable',
                        'is_installed', 'is_returning', 'is_renting', 'last_reported']

availability_df = pd.DataFrame()

for value in dynamic_datapoints:
    availability_df[value] = json_extract(data_dynamic, value)
    
availability_df.head()

,station_id,num_bikes_available,numBikesAvailable,mechanical,ebike,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,213688169,9,9,3,6,25,25,1,1,1,1638195486
1,516709288,0,0,0,0,20,20,1,1,1,1638195109
2,36255,4,4,0,4,16,16,1,1,1,1638195441
3,37815204,5,5,4,1,24,24,1,1,1,1638195239
4,100769544,42,42,31,11,1,1,1,1,1,1638195453


In [29]:
# Merge both datasets

hidalgo_df = velib_df.merge(availability_df, on='station_id', how='left')

hidalgo_df.head()

,station_id,name,lat,lon,capacity,stationCode,num_bikes_available,numBikesAvailable,mechanical,ebike,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,9,9,3,6,25,25,1,1,1,1638195486
1,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,0,0,0,0,20,20,1,1,1,1638195109
2,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,4,4,0,4,16,16,1,1,1,1638195441
3,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,5,5,4,1,24,24,1,1,1,1638195239
4,100769544,Harpe - Saint-Germain,48.851519,2.343670,46,5001,42,42,31,11,1,1,1,1,1,1638195453


### 3 - Paris Map

In [52]:
# Style marker clusters function

icon_create_function="""function(cluster) {
				return L.divIcon({ 
				html: '<div style="width:33px;height:33px;text-align:center;border-radius:40px;font-size:13px;font-weight:700;color:black;background-color:orange">' 
				+ cluster.getChildCount() + '</div>', iconSize: new L.Point(1, 1)});
}"""

In [55]:
# Plot & style Paris map 

paris_map = folium.Map(location=[48.856614, 2.3522219],
                        zoom_start=12, tiles="Stamen Toner")

LocateControl().add_to(paris_map)

locations_stations = hidalgo_df[["lat", "lon"]]
locations_list = locations_stations.values.tolist()

marker_cluster = MarkerCluster(icon_create_function = icon_create_function).add_to(paris_map)

for point in range(0, len(locations_list)):
    folium.Marker(
            location=locations_list[point], 
            popup=folium.Popup((
                    "<strong>" + "Station: " + "</strong>" + str(velib_df["name"][point]) + "<br>" + 
                    "<br>" + "<strong>" + "Mechanical Bikes Available: " + "</strong>" + str(hidalgo_df["mechanical"][point]) + "<br>" + 
                    "<br>" + "<strong>" + "Ebikes Available: " + "</strong>" + str(hidalgo_df["ebike"][point]) + "<br>" + 
                    "<br>" + "<strong>" + "Docks Available: " + "</strong>" + str(hidalgo_df["num_docks_available"][point])), 
                    max_width=180), 
                    icon=folium.Icon(color="red",icon="bicycle", icon_color="white", prefix="fa")).add_to(marker_cluster)

paris_map

In [54]:
# Open map in a web browser

output_file = 'paris_map.html'
paris_map.save(output_file)

webbrowser.open(output_file, new=2)

True